<a href="https://colab.research.google.com/github/GDharan10/Project8_ClassificationModelForFinancialStatements/blob/main/Classification_Model_For_Financial_Statementsipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [2]:
import os
import pandas as pd
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data collection**

In [ ]:
# Function to clean text
def clean_text(text):
    # Remove specified substrings and non-alphanumeric characters, except spaces
    text = re.sub(r'[^a-zA-Z\s]|(?<!\w)(A|B|C|a|b|c|d|e|f|g|h|i|ii|iii|iv|v|vi)(?!\w)', '', text)
    # Remove lowercase letters if they appear separately
    text = re.sub(r'\b[a-z]\b', '', text)
    # Remove uppercase letters if they appear separately
    text = re.sub(r'\b[A-Z]\b', '', text)
    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)
    # Remove special characters
    text = re.sub(r'[^\w\s]+', '', text)
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

    # Remove specified substrings
    substrings_to_remove = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'IX', 'X', 'XIII', 'XIV', 'XV', 'XVI', 'XVII',
                            'XVIII', 'XIX', 'XX', 'XXI', 'XXII', 'XXIII', 'XXIV', 'XXV', 'XXVI']

    for substr in substrings_to_remove:
        text = re.sub(r'\b{}\b'.format(substr), '', text)

In [4]:
# Function to clean the text
def clean_text(text):
    # Remove specified substrings and non-alphanumeric characters, except spaces
    cleaned_text = re.sub(r'[^a-zA-Z\s]|(?<!\w)(A|B|C|a|b|c|d|e|f|g|h|i|ii|iii|iv|v|vi)(?!\w)', '', text)
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# Folder path containing HTML files
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Balance Sheets"

# Initialize an empty list to store DataFrames
dfs_list = []

# Process each HTML file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        file_path = os.path.join(folder_path, filename)
        # Read HTML tables into a list of DataFrame objects
        dfs = pd.read_html(file_path)
        # Assuming the desired table is the first one
        table_df = dfs[0]
        # Extract text from the first two columns of the first table
        extracted_text = ' '.join(str(cell) for cell in table_df.iloc[:, :2].values.flatten() if pd.notnull(cell))
        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)
        # Create a DataFrame with cleaned text and document name
        df = pd.DataFrame({'Extracted_text': [cleaned_text], 'document_name': "Balance Sheets"})
        # Append the DataFrame to the list
        dfs_list.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
BalanceSheets_df = pd.concat(dfs_list, ignore_index=True)

# Display the final DataFrame
print(BalanceSheets_df)


                                        Extracted_text   document_name
0    SI Particulars No No ASSETS NonCurrent Assets ...  Balance Sheets
1    STANDALONE March Particulars Audited ASSETS No...  Balance Sheets
2    GRINDWELL NORTON LIMITED Statement of Standalo...  Balance Sheets
3    ASSETS NonCurrent Assets Property Plant and Eq...  Balance Sheets
4    SIGNED FOR IDENTIFICATIO N CD CD SRBCCO LLP SR...  Balance Sheets
..                                                 ...             ...
265  AUDITED CONSOLIDATED STATEMENT OF ASSETS AND L...  Balance Sheets
266  Notes to standalone I ml AS financial results ...  Balance Sheets
267  Sr No Particulars ASSETS NonCurrent Assets Pro...  Balance Sheets
268  Particulars Particulars ASSETS Noncurrent asse...  Balance Sheets
269  Sr No Particulars Assets NonCurrent Assets Pro...  Balance Sheets

[270 rows x 2 columns]


In [5]:
def clean_text(text):
    # Remove specified substrings and non-alphanumeric characters, except spaces
    text = re.sub(r'[^a-zA-Z\s]|(?<!\w)(A|B|C|a|b|c|d|e|f|g|h|i|ii|iii|iv|v|vi)(?!\w)', '', text)

    # Remove lowercase letters if they appear separately
    text = re.sub(r'\b[a-z]\b', '', text)

    # Remove uppercase letters if they appear separately
    text = re.sub(r'\b[A-Z]\b', '', text)

    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)

    # Remove special characters
    cleaned_text = re.sub(r'[^\w\s]+', '', text)

    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

# Define the folder path
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Cash Flow"

# Initialize an empty list to store DataFrames
dfs_list = []

# Process each HTML file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        # Read HTML file into a DataFrame
        table_df = pd.read_html(os.path.join(folder_path, filename))[0]

        # Extract text from the first two columns of the first table
        extracted_text = ' '.join(str(cell) for cell in table_df.iloc[:, :2].values.flatten() if pd.notnull(cell))

        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)

        # Create a DataFrame with the extracted text and document name
        new_df = pd.DataFrame({'Extracted_text': [cleaned_text], 'document_name': "CashFlow"})

        # Append the DataFrame to the list
        dfs_list.append(new_df)

# Concatenate all DataFrames in the list into a single DataFrame
CashFlow_df = pd.concat(dfs_list, ignore_index=True)

# Display the final DataFrame
print(CashFlow_df)

                                       Extracted_text document_name
0   PARTICULARS As on Net profit before tax ADD De...      CashFlow
1   Cash flows from financing activities Decrease ...      CashFlow
2   Year ended st March CASH FLOW FROM OPERATING A...      CashFlow
3   DESCRIPTION DESCRIPTION Cash Inflow Outflow Fr...      CashFlow
4   INR in Crores Particulars For the year ended M...      CashFlow
5   Particulars Particulars Cash Flow from Operati...      CashFlow
6    Cash flow from operating activities Profit be...      CashFlow
7    CASH FLOW FROM OPERATING ACTIVITIES Net Profi...      CashFlow
8    Cash Flow from Operating Activities Net Profi...      CashFlow
9   Particulars Year ended December Cash flow From...      CashFlow
10  Particulars Cash Flow from operating activitie...      CashFlow
11  As at December CASH FLOW FROM FINANCING ACTIVI...      CashFlow
12   CASH FLOW FROM INVESTING ACTIVITIES CASH FLOW...      CashFlow
13  Year ended st March CASH FLOW FROM FINANCING

In [6]:
# Function to clean text
def clean_text(text):
    # Remove specified substrings
    substrings_to_remove = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'IX', 'X', 'XIII', 'XIV', 'XV', 'XVI', 'XVII',
                            'XVIII', 'XIX', 'XX', 'XXI', 'XXII', 'XXIII', 'XXIV', 'XXV', 'XXVI']

    for substr in substrings_to_remove:
        text = re.sub(r'\b{}\b'.format(substr), '', text)

    # Remove lowercase letters if they appear separately
    text = re.sub(r'\b[a-z]\b', '', text)

    # Remove uppercase letters if they appear separately
    text = re.sub(r'\b[A-Z]\b', '', text)

    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)

    # Remove special characters
    cleaned_text = re.sub(r'[^\w\s]+', '', text)

    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

# Folder path
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Income Statement"

# Initialize an empty list to store dataframes
dfs = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an HTML file
    if filename.endswith(".html"):
        # Read HTML file into a DataFrame
        table_df = pd.read_html(os.path.join(folder_path, filename))[0]

        # Extract text from the first two columns of the first table
        extracted_text = ' '.join(str(cell) for cell in table_df.iloc[:, :2].values.flatten() if pd.notnull(cell))

        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)

        # Create a new DataFrame
        new_df = pd.DataFrame({'Extracted_text': [cleaned_text], 'document_name': "Income Statement"})

        # Append the new DataFrame to the list
        dfs.append(new_df)

# Concatenate all DataFrames in the list
IncomeStatement_df = pd.concat(dfs, ignore_index=True)

# Display the final DataFrame
print(IncomeStatement_df)

                                        Extracted_text     document_name
0    Particulars Income Revenue from Operations Oth...  Income Statement
1    Sr Particular No Income Revenue from operation...  Income Statement
2    Particulars Income Income from operations refe...  Income Statement
3    QUvfttr muled Particulars wined Rrrfi6 Income ...  Income Statement
4    Particulars Audited Particulars Consolidated R...  Income Statement
..                                                 ...               ...
300  SI Particulars No Particulars Revenue from ope...  Income Statement
301  KPlT Technologies Limited Registered Corporate...  Income Statement
302  Finolex Cables Limited Registered Office Mumba...  Income Statement
303  Q4FV18 Revenue Established Service Areas3 Reve...  Income Statement
304  _ _ _ _ _ _ Hi Hn Hi Hn _ _ member of member o...  Income Statement

[305 rows x 2 columns]


In [7]:
def clean_text(text):

    text = re.sub(r'[^a-zA-Z\s]|(?<!\w)(A|B|C|a|b|c|d|e|f|g|h|i|ii|iii|iv|v|vi)(?!\w)', '', text)

    # Remove lowercase letters if they appear separately
    text = re.sub(r'\b[a-z]\b', '', text)

    # Remove uppercase letters if they appear separately
    text = re.sub(r'\b[A-Z]\b', '', text)

    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)

    # Remove special characters
    cleaned_text = re.sub(r'[^\w\s]+', '', text)

    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

# Define the folder path
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Notes"

# Initialize an empty list to store DataFrames
dfs_list = []

# Process each HTML file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        # Read HTML file into a DataFrame
        table_df = pd.read_html(os.path.join(folder_path, filename))[0]

        # Extract text from the first two columns of the first table
        extracted_text = ' '.join(str(cell) for cell in table_df.iloc[:, :2].values.flatten() if pd.notnull(cell))

        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)

        # Create a DataFrame with the extracted text and document name
        new_df = pd.DataFrame({'Extracted_text': [cleaned_text], 'document_name': "Notes"})

        # Append the DataFrame to the list
        dfs_list.append(new_df)

# Concatenate all DataFrames in the list into a single DataFrame
Notes_df = pd.concat(dfs_list, ignore_index=True)

# Display the final DataFrame
print(Notes_df)

                                        Extracted_text document_name
0    Particulars Year ended December Balance at beg...         Notes
1    As at st March Salaries wages bonus and other ...         Notes
2     in Crores in Crores Refer Note Financial liab...         Notes
3    As at Rs Ps Profit After Tax Cr Weighted Avera...         Notes
4     No Particulars No Particulars RESERVES AND SU...         Notes
..                                                 ...           ...
685  Freehold land Cost Cost At April Addition on a...         Notes
686   December INR in Lacs Excise duty on sale of g...         Notes
687  Particulars December Profit for the year Rs in...         Notes
688  NOTE OTHER FINANCIAL ASSETS LONG TERM NONCURRE...         Notes
689   Cash and cash equivalents Cash and cash equiv...         Notes

[690 rows x 2 columns]


In [8]:
def clean_text(text):

    text = re.sub(r'[^a-zA-Z\s]|(?<!\w)(A|B|C|a|b|c|d|e|f|g|h|i|ii|iii|iv|v|vi)(?!\w)', '', text)

    # Remove lowercase letters if they appear separately
    text = re.sub(r'\b[a-z]\b', '', text)

    # Remove uppercase letters if they appear separately
    text = re.sub(r'\b[A-Z]\b', '', text)

    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)

    # Remove special characters
    cleaned_text = re.sub(r'[^\w\s]+', '', text)

    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

# Define the folder path
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Others"

# Initialize an empty list to store DataFrames
dfs_list = []

# Process each HTML file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        # Read HTML file into a DataFrame
        table_df = pd.read_html(os.path.join(folder_path, filename))[0]

        # Extract text from the first two columns of the first table
        extracted_text = ' '.join(str(cell) for cell in table_df.iloc[:, :2].values.flatten() if pd.notnull(cell))

        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)

        # Create a DataFrame with the extracted text and document name
        new_df = pd.DataFrame({'Extracted_text': [cleaned_text], 'document_name': "Others"})

        # Append the DataFrame to the list
        dfs_list.append(new_df)

# Concatenate all DataFrames in the list into a single DataFrame
Others_df = pd.concat(dfs_list, ignore_index=True)

# Display the final DataFrame
print(Others_df)

                                         Extracted_text document_name
0     Consolidated Results Consolidated Results Year...        Others
1                                           QFY QFY RS         Others
2     II Audit Qualification each audit qualificatio...        Others
3      The consolidated financial results of Edelwei...        Others
4     Subsidiaries Subsidiaries Wipro LLC Wipro Gall...        Others
...                                                 ...           ...
1219  Exceptional Items continuing operations Amount...        Others
1220  Particulars Quarter ended Particulars Revenue ...        Others
1221   yL Firms Registration Number Rajiv Singhi Par...        Others
1222    Symbol Typeof security BSE Equity NSE GRINDWELL        Others
1223  Quarter Ended Particulars Mar Total revenues E...        Others

[1224 rows x 2 columns]


In [9]:
df = pd.concat([BalanceSheets_df, CashFlow_df, IncomeStatement_df, Notes_df, Others_df], ignore_index=True)
df

,Extracted_text,document_name
0,SI Particulars No No ASSETS NonCurrent Assets ...,Balance Sheets
1,STANDALONE March Particulars Audited ASSETS No...,Balance Sheets
2,GRINDWELL NORTON LIMITED Statement of Standalo...,Balance Sheets
3,ASSETS NonCurrent Assets Property Plant and Eq...,Balance Sheets
4,SIGNED FOR IDENTIFICATIO N CD CD SRBCCO LLP SR...,Balance Sheets
...,...,...
2520,Exceptional Items continuing operations Amount...,Others
2521,Particulars Quarter ended Particulars Revenue ...,Others
2522,yL Firms Registration Number Rajiv Singhi Par...,Others
2523,Symbol Typeof security BSE Equity NSE GRINDWELL,Others


# **EDA (Exploratory Data Analysis)**

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2525 entries, 0 to 2524
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Extracted_text  2525 non-null   object
 1   document_name   2525 non-null   object
dtypes: object(2)
memory usage: 39.6+ KB


In [11]:
df.describe().T

,count,unique,top,freq
Extracted_text,2525,2413,,24
document_name,2525,5,Others,1224


In [12]:
df.isnull().sum()

Extracted_text    0
document_name     0
dtype: int64

In [13]:
df.duplicated().sum()

109

In [14]:
df.document_name.value_counts()

document_name
Others              1224
Notes                690
Income Statement     305
Balance Sheets       270
CashFlow              36
Name: count, dtype: int64

# **Data_Preprocessing**

In [15]:
df.drop_duplicates(inplace = True)

In [ ]:
df.document_name.value_counts()

document_name
Others              1139
Notes                668
Income Statement     305
Balance Sheets       268
CashFlow              36
Name: count, dtype: int64

In [ ]:
# Function to clean text
def clean_text(text):
    # Strip leading and trailing whitespace
    text = text.strip()
    # Collapse multiple spaces into a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters (keep alphanumeric and spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [ ]:
# Apply the cleaning function to the Extracted_text column
df['Extracted_text'] = df['Extracted_text'].apply(clean_text)

In [16]:
df['Extracted_text'].unique()

array(['SI Particulars No No ASSETS NonCurrent Assets Property Plant and Equipment Capital WorkInProgress Investment Property Goodwill Other Intangible Assets Intangible Assets Under Development Investment in an Associate Financial Assets Investments Loans Others Deferred Tax Assets Net j Other NonCurrent Assets Sub Total NonCurrent Assets Current Assets Inventories Financial Assets Investments Trade Receivables Cash and Cash Equivalents Bank Balance other than above Others Current Tax Assets Net Other Current Assets Sub Total Current Assets TOTAL ASSETS EQUITY AND LIABILITIES EQUITY Equity Share Capital Other Equity Non Controlling Interest Sub Total Equity LIABILITIES NonCurrent Liabilities Financial Liabilities Borrowings Trade Payable Other Financial Liabilities Provisions Deferred Tax Liabilities Net Other NonCurrent Liabilities Sub Total NonCurrent Liabilities Current Liabilities Financial Liabilities Borrowings Trade Payable Other Financial Liabilities Other Current Liabilities 